## 0. Инструменты

Импортируем все необходимое.

In [1]:
import os
import re
import json
import random
import numpy as np
import warnings
from sklearn.decomposition import LatentDirichletAllocation as LDA
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pymystem3 import Mystem
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

## 1. Данные

В качестве корпуса я взяла англоязычный корпус субтитров к реалити-шоу "RuPaul's Drag Race". Он содержит субтитры к 108 эпизодам из 2, 3, 4, 5, 6, 7, 8 и 9 сезонов.

In [2]:
path = '/home/deltamachine/Desktop/projects/rupaul-subtitles-analysis/subtitles/'

season_dirs = os.listdir(path)
corpus = []

for directory in season_dirs:
    episodes = os.listdir(path + directory)
    
    for ep in episodes:
        ep_path = '%s%s/%s' % (path, directory, ep)
        
        with open(ep_path, 'r', encoding="utf-8") as file:
            corpus.append(file.read())

In [3]:
len(corpus)

108

## 2. Препроцессинг
Поступим так:

1) Переведем текст в нижний регистр

2) Выбросим все теги, нечитаемые символы, специфические символы типа ♪, а также отметки времени (корпус - это файлы в формате .srt, т.е. реальные непочищенные субтитры), а также пунктуацию и лишние пробелы

3) Токенизируем и лемматизируем текст.

4) Отсеем стоп-слова, причем не только те, которые нам предлагает NLTK, но и локальные - для этого будем выкидывать 15 самых частотных слов в эпизоде, а также составлять общий список "локальных" стоп-слов. Также добавим к этим стоп-словам небольшой список несвязных междометий, которые, на первый взгляд, часто встречаются в корпусе.

5) Оставим только существительные - при топик моделлинге часто так поступают.

In [4]:
punct = punctuation + '«»—…“”*№–'
all_local_stops = set(['uh', 'mm', 'la', 'yi', 'hmm', 'whoo', 'ah', 'oh', 'hoo', 've', 'ho', 'blah'])
stops = set()

In [5]:
def normalize_text(text, lang):
    global all_local_stops 
    global stops
    
    if lang == 'en':
        stops = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
    elif lang == 'ru':
        stops = set(stopwords.words('russian'))
        lemmatizer = Mystem()
    
    text = text.lower()
    text = re.sub('[0-9]+:[0-9]+.*?\n', ' ', text)
    text = re.sub('<.*?>', ' ', text)
    text = re.sub('\ufeff1', ' ', text)
    text = re.sub('\n[0-9]*', ' ', text)
    text = re.sub('♪', ' ', text)
    text = re.sub('\s{2,}', ' ', text)

    text = [word for word in word_tokenize(text) if word not in punct and word not in stops]  
    
    if lang == 'en': 
        text = [lemmatizer.lemmatize(word) for word in text]
    elif lang == 'ru':
        text = [lemmatizer.lemmatize(word)[0] for word in text]
    
    local_stops = [elem[0] for elem in Counter(text).most_common(15)]
    all_local_stops |= set(local_stops)
    
    tagged_text = pos_tag(text)
    
    text = [word[0] for word in tagged_text if word[0] not in local_stops
                                                     and word[1] == 'NN']
    
    text = list(set(text))
    text = ' '.join(text)

    return text

In [6]:
normalized_corpus = []

for text in corpus:
    normalized_text = normalize_text(text, 'en')
    normalized_corpus.append(normalized_text)

## 3. Topic modelling: cпособ 1

Сначала попробуем использовать TfIdf в сочетании с латентным размещением Дирихле из пакета sklearn.

In [7]:
stops |= all_local_stops

In [8]:
vectorizer = TfidfVectorizer(stop_words=stops)
matrix = vectorizer.fit_transform(normalized_corpus)
words = vectorizer.get_feature_names()

In [9]:
warnings.simplefilter("ignore", DeprecationWarning)

In [10]:
def topic_modelling_sklearn(vectorizer, matrix, topics_num, words_num):
    lda = LDA(n_components=topics_num, n_jobs=-1, learning_method='online')
    lda.fit(matrix)

    for topic_index, topic in enumerate(lda.components_):
        print('\nTopic %s:' % str(topic_index + 1))
        print(' '.join([words[i] for i in topic.argsort()[:-words_num - 1:-1]]))

Попробуем выделить 10 тем.

In [15]:
topics_num = 10
words_num = 7

topic_modelling_sklearn(vectorizer, matrix, topics_num, words_num)


Topic 1:
bulge mortgage chill constituency sarge woman river

Topic 2:
lip okay hi decision kind man body

Topic 3:
roulez bed husband amy tchotchke grocery snake

Topic 4:
lip god week cover side thought someone

Topic 5:
launch chance painting slit hutton tina dreamt

Topic 6:
tone terrible level binge nobody drinking group

Topic 7:
hen whory breathy sleeve rescue mix triple

Topic 8:
body knee anything beyoncé hope gayestballever construction

Topic 9:
poker coal stepsister burn comfortable summer jordin

Topic 10:
cue heterosexuality pitcher hardwood tell sunset fixin


Не особо вижу смысл.

Попробуем выделить 3 темы.

In [17]:
topics_num = 3
words_num = 7

topic_modelling_sklearn(vectorizer, matrix, topics_num, words_num)


Topic 1:
man santino represent word cocktail woman club

Topic 2:
hi lip okay body decision man kind

Topic 3:
runway mr model supply face cover serve


Последняя тема в принципе содержит в себе лексику, использующуюся во время презентаций нарядов на подиуме (runway, face, serve, "Cover Girl" - название песни, которая часто играет на подиуме), но первые две темы как-то смысла не несут.

...или 25.

In [20]:
topics_num = 25
words_num = 7

topic_modelling_sklearn(vectorizer, matrix, topics_num, words_num)


Topic 1:
crip miranda resurrection countless salt legacy allude

Topic 2:
bot ageless motel sarah follicle clothing albert

Topic 3:
hug bye pride president parent nail lip

Topic 4:
sober corset ruco scarf ex minj dance

Topic 5:
boo please fire supply engine charisma lifetime

Topic 6:
creator oscar masculine carrera ruffle tatianna wink

Topic 7:
vacuum living baloney self twice film budget

Topic 8:
trinity bag damn duck city date business

Topic 9:
jewel network audience element television righty volume

Topic 10:
shoot capulet flaunt kat bird ruche disaster

Topic 11:
kamaru rachael jockey parent dryer eloquent performing

Topic 12:
bitchfest leash else silicone facet quick lone

Topic 13:
lan cigarette girlfriend essence lol gate scene

Topic 14:
scale demand peron wad travel squeak pa

Topic 15:
impersonation buff concert doubt wootity silicone getup

Topic 16:
davenport exit bed apparent smudge frail heart

Topic 17:
gender eyelash happen conversation doll amy goyls

Topic 18

Все по-прежнему плохо, но уже чуть интереснее. Тема 21 выглядит как звукоподражания ('dun', 'dah', 'gah', 'boom'). Тема 5 собрала в себе кусочки фраз, повторяющихся в каждом эпизоде: lifetime supply (of cosmetics) в качестве одного из призов победителю, "charisma, uniqueness, nerve and talent" и "gentlemen, start your engines!". Тема 9 содержит лексику, употребляющуюся на финалах (там и network, и audience, и транслируется все на television, и призы от компании Fierce Drag Jewels).

## 4. Topic modelling: способ 2

Сейчас попробуем использовать инструменты из пакета gensim - там есть свой алгоритм латентного размещения Дирихле.

In [21]:
normalized_corpus_lda = [elem.split() for elem in normalized_corpus]

In [22]:
dictionary = Dictionary(normalized_corpus_lda)
lda_corpus = [dictionary.doc2bow(text) for text in normalized_corpus_lda]

In [25]:
def topic_modelling_gensim(lda_corpus, dictionary, topics_num, words_num):
    lda = LdaModel(lda_corpus, id2word=dictionary, num_topics=topics_num)

    topics = lda.show_topics(num_topics=topics_num, num_words=words_num, formatted=False)
    topics = [(topic[0], [word[0] for word in topic[1]]) for topic in topics]

    for num, words in topics:
        print('\nTopic %s:' % (str(num)))
        print(' '.join(words))

Сначала на 10 темах.

In [26]:
topics_num = 10
words_num = 7

topic_modelling_gensim(lda_corpus, dictionary, topics_num, words_num)


Topic 0:
competition name stage anything time today chance

Topic 1:
god music woman decision feel hello competition

Topic 2:
something winner thing challenge engine guy woman

Topic 3:
day today thing part face lady game

Topic 4:
need hello let way guy winner week

Topic 5:
kind head everyone moment everything hair runway

Topic 6:
lady work cash mean somebody feel something

Topic 7:
judge show stage bit decision way talk

Topic 8:
bit head today work life luck elimination

Topic 9:
let something face home hey lot bit


Кое-что можно выделить. Этот алгоритм лучше выцепляет фразы, которые повторяются из эпизода в эпизод в одних и тех же контекстах и вообще вполне себе репрезентуют определенные темы.

Например, тема победы в третьем топике - там разные куски из фразы "Gentlemen, start your engines and may the best woman win!".

Тема судейства в седьмом топике - также куски типичных фраз "While you untuck backstage, the judges and I will deliberate"  и "I made my decision".

Тема финального испытания каждого эпизода, в котором определяется, кто из королев уйдет, в 8 топике - также угадываются типичные для этого контекста фразы "save yourself from elimination", "lipsync for your life", "good luck and don't fuck it up".

Попробуем 3 темы.

In [27]:
topics_num = 3
words_num = 7

topic_modelling_gensim(lda_corpus, dictionary, topics_num, words_num)


Topic 0:
let way moment winner hello bit music

Topic 1:
something week life competition woman race honey

Topic 2:
home challenge thing year day woman winner


...нет, не особо осмысленно.

Попробуем 25 тем.

In [28]:
topics_num = 25
words_num = 7

topic_modelling_gensim(lda_corpus, dictionary, topics_num, words_num)


Topic 0:
time woman winner hey way elimination let

Topic 1:
show everybody visage lady world thank kind

Topic 2:
work time winner face everybody hello home

Topic 3:
hell honey challenge hair face queen god

Topic 4:
day body time lady challenge ru honey

Topic 5:
hell bitch chance way hi hair judge

Topic 6:
competition home cash room day kind woman

Topic 7:
mean cash anything guy time man kind

Topic 8:
show guy thing hey hi life race

Topic 9:
life way somebody judge something music guy

Topic 10:
work race face music stage lot engine

Topic 11:
judge woman challenge feel thing lady life

Topic 12:
face stage world thank see music anything

Topic 13:
woman judge year deliberate talk today body

Topic 14:
lot show word bitch hello engine judge

Topic 15:
everything honey head runway somebody music day

Topic 16:
everything thank home week feel time day

Topic 17:
winner okay kind luck god impress moment

Topic 18:
life face feel week yeah tonight music

Topic 19:
winner today kin

...а вот это уже малоосмысленно, видимо, слишком большое количество тем. Можно с натяжкой что-то повыделять, но это будет не так красиво, как в случае с 10 темами.